In [23]:
import pandas as pd
import numpy as np
import os

In [2]:
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')    

In [3]:
test = test[(test['type'] == 'Assessment') & (((test['event_code'] == 4100) & (test['title'] != 'Bird Measurer (Assessment)')) | ((test['event_code'] == 4110) & (test['title'] == 'Bird Measurer (Assessment)')))]

(4492, 11)

In [ ]:
test = test.reset_index(drop=False)
test.drop(columns = ['index'],axis = 1, inplace = True)

In [4]:
session_count = test['game_session'].value_counts().to_dict()
test['assessment_attempt_count'] = test['game_session'].map(session_count)

In [5]:
test['contains_true_assessment'] = test['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

In [6]:
change_value = {
    True : 1,
    False : 0
}
test['contains_true_assessment'] = test['contains_true_assessment'].map(change_value)

In [7]:
correct_attempt = dict(test.groupby('game_session',sort=False)['contains_true_assessment'].sum())
test['contains_true_assessment_count'] = test['game_session'].map(correct_attempt)

In [8]:
for c in ['contains_true_assessment']:
    test.pop(c)

In [11]:
test['accumulated_accuracy'] = np.where((test['contains_true_assessment_count'] == 0),0,(test['contains_true_assessment_count']/test['assessment_attempt_count']))

In [12]:
test.loc[(test['type'] == 'Assessment'), 'accuracy_group'] = 0
test.loc[(test['accumulated_accuracy'] == 1) & (test['type'] == 'Assessment'), 'accuracy_group'] = 3
test.loc[(test['accumulated_accuracy'] == 0.5) & (test['type'] == 'Assessment'), 'accuracy_group'] = 2
test.loc[(test['accumulated_accuracy'] < 0.5) & (test['accumulated_accuracy'] > 0) & (test['assessment_attempt_count'] > 0) & (test['type'] == 'Assessment'), 'accuracy_group'] = 1

In [13]:
test.rename(columns = {'contains_true_assessment_count': 'num_correct',
                        'accumulated_accuracy':'accuracy',
                        'assessment_attempt_count': 'total_attempt'},inplace=True)

In [14]:
test['num_incorrect'] = test['total_attempt'] - test['num_correct']

In [15]:
test = test[['event_id','game_session','timestamp','event_data','installation_id','event_count','game_time','event_code','title','type','world','num_correct','num_incorrect','total_attempt','accuracy','accuracy_group']]

In [16]:
for c in ['event_id','timestamp','event_data','event_code','event_count','type']:
    test.pop(c)

In [17]:
result = dict(test.groupby('installation_id')['accuracy_group'].last())


{'00abaee7': 3.0,
 '01242218': 1.0,
 '02256298': 3.0,
 '027e7ce5': 3.0,
 '02a29f99': 2.0,
 '03e33699': 3.0,
 '04d31500': 3.0,
 '0512bf0e': 3.0,
 '05771bba': 0.0,
 '05b82cf5': 3.0,
 '0617500d': 3.0,
 '07749e99': 3.0,
 '08611cc8': 0.0,
 '090fe325': 3.0,
 '0937340d': 2.0,
 '09aaaf83': 1.0,
 '09aefe80': 3.0,
 '0a126293': 1.0,
 '0a2a77b2': 1.0,
 '0b24b6ac': 2.0,
 '0b607c82': 3.0,
 '0d735146': 3.0,
 '0d7752d3': 3.0,
 '0dd670e9': 0.0,
 '0e514571': 3.0,
 '0e718764': 0.0,
 '0ea27b66': 0.0,
 '0f7116a6': 0.0,
 '101d16f5': 1.0,
 '10acf963': 2.0,
 '1121f331': 0.0,
 '1181ce7c': 1.0,
 '12771ee9': 1.0,
 '13629687': 3.0,
 '13bcaf23': 0.0,
 '140087ce': 3.0,
 '1594c19e': 1.0,
 '15ec4544': 3.0,
 '15f9b137': 1.0,
 '16160dde': 2.0,
 '163ffbd7': 3.0,
 '18569185': 0.0,
 '18a43ba3': 2.0,
 '18dd112c': 3.0,
 '1962067f': 3.0,
 '1a20a79d': 3.0,
 '1b38b81a': 0.0,
 '1b3987c8': 1.0,
 '1b5d84df': 2.0,
 '1b6ecb54': 3.0,
 '1baa3c99': 3.0,
 '1d2eae73': 0.0,
 '1dd31d7c': 0.0,
 '1df0bdd4': 3.0,
 '1e8622da': 3.0,
 '1ed372f8

In [20]:
submission['accuracy_group'] = submission['installation_id'].map(result)

In [21]:
submission.to_csv('submission.csv', index=None)

In [ ]:
submission['accuracy_group'].isnull().sum()